<a href="https://colab.research.google.com/github/kbsmd-sportsmusicdata/Basketball-Analytics-Course/blob/main/GetTotals_pbpstats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import annotations

import json
from pathlib import Path
from datetime import datetime, timezone
import requests

def fetch_pbpstats_totals_wnba(
    seasons: str = "2023,2024,2025",
    season_type: str = "Regular Season",
    data_type: str = "Player",
    timeout: int = 60
) -> dict:
    url = "https://api.pbpstats.com/get-totals/wnba"
    params = {
        "Season": seasons,
        "SeasonType": season_type,
        "Type": data_type
    }
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json()

def save_raw_json(payload: dict, output_dir: str | Path) -> tuple[Path, Path]:
    """
    Saves:
      1) timestamped file (immutable snapshot)
      2) latest.json (overwritten each run)
    Returns (snapshot_path, latest_path)
    """
    output_dir = Path(output_dir).expanduser().resolve()
    output_dir.mkdir(parents=True, exist_ok=True)

    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%SZ")
    snapshot_path = output_dir / f"pbpstats_totals_wnba_player_{ts}.json"
    latest_path = output_dir / "pbpstats_totals_wnba_player_latest.json"

    # write snapshot
    snapshot_path.write_text(
        json.dumps(payload, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # write latest
    latest_path.write_text(
        json.dumps(payload, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    return snapshot_path, latest_path

if __name__ == "__main__":
    payload = fetch_pbpstats_totals_wnba(
        seasons="2023,2024,2025",
        season_type="Regular Season",
        data_type="Player"
    )

    # ✅ Option A: if you run this *from within the repo*, use a relative path:
    # repo_relative_dir = Path("scripts/wnba_scripts/pbpstats/raw_json_data")

    # ✅ Option B: hard-point to your local repo clone (edit this once):
    repo_relative_dir = Path("/content/Basketball-Analytics-Course/scripts/wnba_scripts/pbpstats/raw_json_data")

    snap, latest = save_raw_json(payload, repo_relative_dir)

    # Access the table data if you want it after saving:
    player_stats = payload.get("multi_row_table_data", [])
    print(f"Saved raw JSON snapshot: {snap}")
    print(f"Saved raw JSON latest:   {latest}")
    print(f"multi_row_table_data rows: {len(player_stats)}")


Saved raw JSON snapshot: /content/Basketball-Analytics-Course/scripts/wnba_scripts/pbpstats/raw_json_data/pbpstats_totals_wnba_player_20251225_194555Z.json
Saved raw JSON latest:   /content/Basketball-Analytics-Course/scripts/wnba_scripts/pbpstats/raw_json_data/pbpstats_totals_wnba_player_latest.json
multi_row_table_data rows: 257
